In [21]:
import torch, torchvision
from mobilenetv3 import MobileNetV3_forFPN, MobileNetV3, load_pretrained_fpn
from tests import test_loaded_weights, compare_output
%load_ext autoreload
%autoreload 2
#torch.backends.cudnn.deterministic = True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
mn3_fpn = MobileNetV3_forFPN()
mn3_fpn = load_pretrained_fpn(mn3_fpn,\
                              '/home/davidyuk/Projects/backbones/pytorch-mobilenet-v3/mobilenetv3_small_67.4.pth.tar')
with torch.no_grad():
    print(mn3_fpn.forward(torch.zeros(1,3,224,224)).size())
# tuple(mn3_fpn.parameters())

torch.Size([1, 96, 7, 7])


In [5]:
test_loaded_weights()

test passed


In [7]:
for name, parameter in mn3_fpn.named_parameters():
    print(name)

layers_os4.0.0.weight
layers_os4.0.1.weight
layers_os4.0.1.bias
layers_os4.1.conv.0.weight
layers_os4.1.conv.1.weight
layers_os4.1.conv.1.bias
layers_os4.1.conv.3.weight
layers_os4.1.conv.4.weight
layers_os4.1.conv.4.bias
layers_os4.1.conv.5.fc.0.weight
layers_os4.1.conv.5.fc.2.weight
layers_os4.1.conv.7.weight
layers_os4.1.conv.8.weight
layers_os4.1.conv.8.bias
layers_os8.0.conv.0.weight
layers_os8.0.conv.1.weight
layers_os8.0.conv.1.bias
layers_os8.0.conv.3.weight
layers_os8.0.conv.4.weight
layers_os8.0.conv.4.bias
layers_os8.0.conv.7.weight
layers_os8.0.conv.8.weight
layers_os8.0.conv.8.bias
layers_os8.1.conv.0.weight
layers_os8.1.conv.1.weight
layers_os8.1.conv.1.bias
layers_os8.1.conv.3.weight
layers_os8.1.conv.4.weight
layers_os8.1.conv.4.bias
layers_os8.1.conv.7.weight
layers_os8.1.conv.8.weight
layers_os8.1.conv.8.bias
layers_os16.0.conv.0.weight
layers_os16.0.conv.1.weight
layers_os16.0.conv.1.bias
layers_os16.0.conv.3.weight
layers_os16.0.conv.4.weight
layers_os16.0.conv.4.bi

In [33]:
from torchvision.models.detection.backbone_utils import BackboneWithFPN

def mobilenetV3_fpn_backbone(pretrained=False):
    backbone = MobileNetV3_forFPN()
    
    if pretrained:
        load_pretrained_fpn(backbone,\
                              '/home/davidyuk/Projects/backbones/pytorch-mobilenet-v3/mobilenetv3_small_67.4.pth.tar')
        
    return_layers = {'layers_os4': 0, 'layers_os8': 1, 'layers_os16': 2, 'layers_os32': 3}

    in_channels_list = [16, 24, 48, 96]
    out_channels = 100
    return BackboneWithFPN(backbone, return_layers, in_channels_list, out_channels)

In [36]:
mb3_ = mobilenetV3_fpn_backbone(pretrained=True)
mb3_.forward(torch.ones(1,3,224,224))[0].size()

torch.Size([1, 100, 56, 56])

In [ ]:
backbone (nn.Module): the network used to compute the features for the model.
    It should contain a out_channels attribute, which indicates the number of output
    channels that each feature map has (and it should be the same for all feature maps).
    The backbone should return a single Tensor or and OrderedDict[Tensor].
num_classes (int): number of output classes of the model (including the background).
    If box_predictor is specified, num_classes should be None.
min_size (int): minimum size of the image to be rescaled before feeding it to the backbone
max_size (int): maximum size of the image to be rescaled before feeding it to the backbone
image_mean (Tuple[float, float, float]): mean values used for input normalization.
    They are generally the mean values of the dataset on which the backbone has been trained
    on
image_std (Tuple[float, float, float]): std values used for input normalization.
    They are generally the std values of the dataset on which the backbone has been trained on
rpn_anchor_generator (AnchorGenerator): module that generates the anchors for a set of feature
    maps.
rpn_head (nn.Module): module that computes the objectness and regression deltas from the RPN
rpn_pre_nms_top_n_train (int): number of proposals to keep before applying NMS during training
rpn_pre_nms_top_n_test (int): number of proposals to keep before applying NMS during testing
rpn_post_nms_top_n_train (int): number of proposals to keep after applying NMS during training
rpn_post_nms_top_n_test (int): number of proposals to keep after applying NMS during testing
rpn_nms_thresh (float): NMS threshold used for postprocessing the RPN proposals
rpn_fg_iou_thresh (float): minimum IoU between the anchor and the GT box so that they can be
    considered as positive during training of the RPN.
rpn_bg_iou_thresh (float): maximum IoU between the anchor and the GT box so that they can be
    considered as negative during training of the RPN.
rpn_batch_size_per_image (int): number of anchors that are sampled during training of the RPN
    for computing the loss
rpn_positive_fraction (float): proportion of positive anchors in a mini-batch during training
    of the RPN
box_roi_pool (MultiScaleRoIAlign): the module which crops and resizes the feature maps in
    the locations indicated by the bounding boxes
box_head (nn.Module): module that takes the cropped feature maps as input
box_predictor (nn.Module): module that takes the output of box_head and returns the
    classification logits and box regression deltas.
box_score_thresh (float): during inference, only return proposals with a classification score
    greater than box_score_thresh
box_nms_thresh (float): NMS threshold for the prediction head. Used during inference
box_detections_per_img (int): maximum number of detections per image, for all classes.
box_fg_iou_thresh (float): minimum IoU between the proposals and the GT box so that they can be
    considered as positive during training of the classification head
box_bg_iou_thresh (float): maximum IoU between the proposals and the GT box so that they can be
    considered as negative during training of the classification head
box_batch_size_per_image (int): number of proposals that are sampled during training of the
    classification head
box_positive_fraction (float): proportion of positive proposals in a mini-batch during training
    of the classification head
bbox_reg_weights (Tuple[float, float, float, float]): weights for the encoding/decoding of the
    bounding boxes
mask_roi_pool (MultiScaleRoIAlign): the module which crops and resizes the feature maps in
     the locations indicated by the bounding boxes, which will be used for the mask head.
mask_head (nn.Module): module that takes the cropped feature maps as input
mask_predictor (nn.Module): module that takes the output of the mask_head and returns the
    segmentation mask logits

In [ ]:
from torchvision.models.detection.mask_rcnn import MaskRCNN
def maskrcnn_mobileNetV3_fpn(num_classes=3, pretrained_backbone=True):
    